### **DAY 9 (17/01/26) – SQL Analytics & Dashboards**

### Learn:

- SQL warehouses
- Complex analytical queries
- Dashboard creation
- Visualizations & filters

### 🛠️ Tasks:

1. Create SQL warehouse
2. Write analytical queries
3. Build dashboard: revenue trends, funnels, top products
4. Add filters & schedule refresh

In [0]:
%sql
WITH daily AS (
  SELECT event_date, SUM(price) as rev
  FROM ecommerce.silver.events GROUP BY event_date
)
SELECT event_date, rev,
  AVG(rev) OVER (ORDER BY event_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as moving_avg
FROM daily;

In [0]:
%sql
-- Conversion funnel
SELECT category_code,
  sum(Case when event_type = 'view' then 1 else 0 end)as views_count,
  sum(Case when event_type = 'purchase' then 1 else 0 end)as purchases,
  ROUND(sum(Case when event_type = 'purchase' then 1 else 0 end)*100.0/sum(Case when event_type = 'view' then 1 else 0 end), 2) as conversion_rate
FROM ecommerce.silver.events
where category_code is not null
GROUP BY category_code
order by conversion_rate desc
limit 5;


In [0]:
%sql
-- Customer tiers
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM ecommerce.silver.events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier
order by avg_ltv desc;
